<a href="https://colab.research.google.com/github/Nuha4/adelaide_metrocard-cpi-fuel-correlation/blob/main/adl_metrocard_cpi_fuel_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
import numpy as np
import glob
import os

In [ ]:
# Load, Merge and Clean MetroCard Usage Data for Adelaide (2015_Q4 to 2024_Q4)
folder_path = 'data_metro/'
file_pattern = os.path.join(folder_path, "*anded*.csv")
all_files = glob.glob(file_pattern)

# Read and concatenate
df_list = []
for file in all_files:
    df = pd.read_csv(file)
    df['SOURCE_FILE'] = os.path.basename(file)  # Optional: track source file
    df_list.append(df)
combined_metro_df = pd.concat(df_list, ignore_index=True)

print(combined_metro_df)

# # Check unparseable rows
# invalid_dates = combined_metro_df[combined_metro_df['VALIDATION_DATE'].isna()]
# print("Unparseable rows:", len(invalid_dates))
# print(invalid_dates[['SOURCE_FILE', 'VALIDATION_DATE']].head())


# Convert date and prepare monthly data
combined_metro_df['VALIDATION_DATE'] = pd.to_datetime(
    combined_metro_df['VALIDATION_DATE'],
    format='mixed',
    dayfirst=True,
    errors='coerce'
)
# combined_metro_df['VALIDATION_DATE'] = pd.to_datetime(combined_metro_df['VALIDATION_DATE'], dayfirst=True)
combined_metro_df['BAND_BOARDINGS_FLOOR'] = pd.to_numeric(combined_metro_df['BAND_BOARDINGS_FLOOR'], errors='coerce')
combined_metro_df['Month'] = combined_metro_df['VALIDATION_DATE'].dt.to_period('M').dt.to_timestamp()

# Aggregate by month
monthly_metro = combined_metro_df.groupby('Month')['BAND_BOARDINGS_FLOOR'].sum().reset_index()
monthly_metro = monthly_metro.rename(columns={'BAND_BOARDINGS_FLOOR': 'MetroCard_Usage'})

# Save for reuse
monthly_metro.to_csv("combined_metrocard_monthly.csv", index=False)
print(monthly_metro.head())


         VALIDATION_DATE  NUM_MODE_TRANSPORT ROUTE_CODE  ROUTE_DIRECTION  \
0             22/05/2023                   0          0                0   
1             19/06/2023                   0          0                0   
2              2/04/2023                   1     OD1MTB                0   
3              4/05/2023                   1     OD1MTB                0   
4             13/05/2023                   1     OD1MTB                0   
...                  ...                 ...        ...              ...   
59247577      11/04/2024                   1         J1                1   
59247578      11/04/2024                   1         J2                1   
59247579      11/04/2024                   1        M44                1   
59247580      11/04/2024                   1       178M                1   
59247581      11/04/2024                   1        252                1   

           GTFS_ID  MEDIUM_TYPE BAND_BOARDINGS  BAND_BOARDINGS_FLOOR  \
0         10184

In [35]:
# Drop extra rows after 2024 Q4
monthly_metro = pd.read_csv('combined_metrocard_monthly.csv')
monthly_metro = monthly_metro[monthly_metro['Month'] <= '2024-12-31']
print(monthly_metro.head())
print(monthly_metro.tail())


        Month  MetroCard_Usage
0  2015-10-01          5242675
1  2015-11-01          5127222
2  2015-12-01          4052606
3  2016-01-01          3721365
4  2016-02-01          5220318
          Month  MetroCard_Usage
106  2024-08-01          4347875
107  2024-09-01          4032887
108  2024-10-01          4046780
109  2024-11-01          3901431
110  2024-12-01          3056540


In [37]:
summary = monthly_metro['MetroCard_Usage'].describe().astype(int).apply(lambda x: f"{x:,}")
print(summary)

count          111
mean     4,173,776
std      1,164,770
min        965,965
25%      3,442,987
50%      4,135,112
75%      5,035,557
max      6,354,670
Name: MetroCard_Usage, dtype: object


In [38]:
# Load and Clean CPI Data (2015 Q4 to 2024 Q4)
cpi_preview = pd.read_excel("CPI_640103.xlsx", sheet_name='Data1', header=None)
cpi_data = cpi_preview.iloc[10:, [0, 4]].copy()
cpi_data.columns = ['Date', 'CPI_Adelaide']

# Convert to datetime and numeric
cpi_data['Date'] = pd.to_datetime(cpi_data['Date'], errors='coerce')
cpi_data['CPI_Adelaide'] = pd.to_numeric(cpi_data['CPI_Adelaide'], errors='coerce')

# Drop rows with missing dates and Filter
cpi_data = cpi_data[cpi_data['Date'].notna()]
cpi_data = cpi_data[(cpi_data['Date'] >= '2015-07-01') & (cpi_data['Date'] <= '2024-12-31')]

# Interpolate quarterly data to monthly
cpi_monthly = cpi_data.set_index('Date').resample('MS').interpolate(method='linear').reset_index()
cpi_monthly['Month'] = cpi_monthly['Date'].dt.to_period('M').dt.to_timestamp()

In [52]:
# Drop extra rows before 2015 Q4
cpi_monthly = cpi_monthly[cpi_monthly['Month'] >= '2015-10-01']
print(cpi_monthly.head())
print(cpi_monthly.tail())


        Date  CPI_Adelaide      Month
1 2015-10-01    115.200000 2015-10-01
2 2015-11-01    115.300000 2015-11-01
3 2015-12-01    115.400000 2015-12-01
4 2016-01-01    115.566667 2016-01-01
5 2016-02-01    115.733333 2016-02-01
          Date  CPI_Adelaide      Month
107 2024-08-01    152.833333 2024-08-01
108 2024-09-01    153.400000 2024-09-01
109 2024-10-01    152.600000 2024-10-01
110 2024-11-01    151.800000 2024-11-01
111 2024-12-01    151.000000 2024-12-01


In [53]:
# Load and filter SA fuel consumption data  (2015_Q4 to 2024_Q4)
fuel_df = pd.read_excel("australian-petroleum-statistics-data-extract-december-2024.xlsx",
                        sheet_name='Sales by state and territory')

fuel_df = fuel_df[(fuel_df['Month'] >= '2015-10-01') & (fuel_df['Month'] <= '2024-12-31')]
fuel_df = fuel_df[fuel_df['State'] == 'SA'].copy()

# Clean and format columns
fuel_df['Month'] = pd.to_datetime(fuel_df['Month'], errors='coerce')
fuel_df['Automotive gasoline: total (ML)'] = fuel_df['Automotive gasoline: total (ML)'].replace('n.p.', np.nan)
fuel_df['Automotive gasoline: total (ML)'] = pd.to_numeric(fuel_df['Automotive gasoline: total (ML)'], errors='coerce')

# Finalize structure
fuel_df = fuel_df[['Month', 'Automotive gasoline: total (ML)']]
fuel_df = fuel_df.rename(columns={'Automotive gasoline: total (ML)': 'Fuel_Consumption_ML'})

print(fuel_df.head())
print(fuel_df.tail())



         Month  Fuel_Consumption_ML
585 2015-10-01                106.2
586 2015-11-01                102.8
587 2015-12-01                115.1
588 2016-01-01                101.0
589 2016-02-01                104.8
         Month  Fuel_Consumption_ML
691 2024-08-01                 89.2
692 2024-09-01                 84.8
693 2024-10-01                 86.7
694 2024-11-01                 88.9
695 2024-12-01                 85.9


In [62]:
cpi_monthly['Month'] = pd.to_datetime(cpi_monthly['Month'])
fuel_df['Month'] = pd.to_datetime(fuel_df['Month'])
monthly_metro['Month'] = pd.to_datetime(monthly_metro['Month'])

# Merge All Datasets
combined_df = pd.merge(cpi_monthly[['Month', 'CPI_Adelaide']], fuel_df, on='Month', how='outer')
combined_df = pd.merge(combined_df, monthly_metro, on='Month', how='outer')
combined_df = combined_df.sort_values('Month').reset_index(drop=True)

# Save for resue
combined_df.to_csv("metrocard_cpi_fuel_combined.csv", index=False)
print(combined_df.head())
print(combined_df.tail())

       Month  CPI_Adelaide  Fuel_Consumption_ML  MetroCard_Usage
0 2015-10-01    115.200000                106.2          5242675
1 2015-11-01    115.300000                102.8          5127222
2 2015-12-01    115.400000                115.1          4052606
3 2016-01-01    115.566667                101.0          3721365
4 2016-02-01    115.733333                104.8          5220318
         Month  CPI_Adelaide  Fuel_Consumption_ML  MetroCard_Usage
106 2024-08-01    152.833333                 89.2          4347875
107 2024-09-01    153.400000                 84.8          4032887
108 2024-10-01    152.600000                 86.7          4046780
109 2024-11-01    151.800000                 88.9          3901431
110 2024-12-01    151.000000                 85.9          3056540


In [63]:
# Data Quality Check
print(combined_df.dtypes)
print(combined_df.isna().sum())
missing_rows = combined_df[combined_df.isna().any(axis=1)]
print(missing_rows)
print(combined_df.duplicated().sum())
print(combined_df['Month'].min(), combined_df['Month'].max())
print(combined_df.describe())

Month                  datetime64[ns]
CPI_Adelaide                  float64
Fuel_Consumption_ML           float64
MetroCard_Usage                 int64
dtype: object
Month                  0
CPI_Adelaide           0
Fuel_Consumption_ML    0
MetroCard_Usage        0
dtype: int64
Empty DataFrame
Columns: [Month, CPI_Adelaide, Fuel_Consumption_ML, MetroCard_Usage]
Index: []
0
2015-10-01 00:00:00 2024-12-01 00:00:00
                               Month  CPI_Adelaide  Fuel_Consumption_ML  \
count                            111    111.000000           111.000000   
mean   2020-05-01 10:48:38.918918912    129.733333            94.717117   
min              2015-10-01 00:00:00    115.200000            60.800000   
25%              2018-01-16 12:00:00    123.716667            87.550000   
50%              2020-05-01 00:00:00    125.933333            94.600000   
75%              2022-08-16 12:00:00    137.366667           102.050000   
max              2024-12-01 00:00:00    153.400000         